# 2021 Intro. to Machine Learning 
## Program Assignment #2 - Decision Tree & Random Forest & KNN & PCA


### 0816153 陳琮方

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from sklearn import model_selection
from sklearn.model_selection import train_test_split, KFold

# Data Input & Preprocessing

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

data_path = './Dataset/student-mat.csv'
TARGET_LABEL = 'G3'

In [ ]:
raw_data = pd.read_csv(data_path, sep = ';')

print(f'Dataset size = {raw_data.shape}\n')

X_raw = raw_data.drop(TARGET_LABEL, axis = 1)
Y_raw = raw_data[TARGET_LABEL]

print(f'X size: {X_raw.shape}')

### OneHotEncode

In [ ]:
# Binary Features
bin_feat = [
    'school', 'sex', 'address', 'famsize', 'Pstatus', 
    'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 
    'internet', 'romantic'
]

# Nominal Features
cat_feat = [
    'Mjob', 'Fjob', 'reason', 'guardian'           
]

In [ ]:
enc = OneHotEncoder(categories = 'auto', drop='if_binary')
enc.fit(X_raw[cat_feat])

data_array = enc.transform(X_raw[cat_feat]).toarray()
feat_names = enc.get_feature_names_out(cat_feat)

data_frams = pd.DataFrame(data_array, index = X_raw.index, columns = feat_names)

X = pd.concat([X_raw.drop(columns = cat_feat), data_frams], axis = 1)

print(f'Number of categorical features: {len(cat_feat)}')
print(f'Sum of unique categorical: {len(feat_names)}')

print('\n=====================================')
print(f'X size after one-hot: {X.shape}')

### Other Preprocessing

- Binary classification - pass if G3 ≥ 10, else fail.
- 5-Level classification - based on the Erasmus grad conversion system.


|G3   | 16-20  | 14-15  | 12-13  |  10-11 |   0-9  |
|-----|--------|--------|--------|--------|--------|
|Rank |    A   |    B   |    C   |    D   |    F   |

In [ ]:
Y_bin = []
Y_5lv = []

for i in Y_raw:
    Y_bin.append((1 if i >= 10 else 0))
    if i >= 16: Y_5lv.append('A'); continue;
    if i >= 14: Y_5lv.append('B'); continue;
    if i >= 12: Y_5lv.append('C'); continue;
    if i >= 10: Y_5lv.append('D'); continue;
    Y_5lv.append('F')

# Principal Components Analysis (PCA)

In [ ]:
from sklearn import decomposition

PCA_DIMENTION = 15

In [ ]:
pca = decomposition.PCA(n_components = PCA_DIMENTION)
pca.fit(X)

print(pca.components)